In [1]:
import requests as r
import os
import pandas as pd
from pandas import json_normalize
import json
from itertools import product

In [2]:
#Schema das Tabelas

data_types_dict_orcamentaria = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'str', 'funcao': 'int', 'subfuncao': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'educacao_saude': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'natureza_despesa': 'str', 'ano_inscricao': 'int', 'natureza_receita': 'str', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}
data_types_dict_patrimonial = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'financeiro_permanente': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'divida_consolidada': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}
data_types_dict_controle = {'tipo_matriz': 'str', 'cod_ibge': 'str', 'classe_conta': 'int', 'conta_contabil': 'str', 'poder_orgao': 'int', 'ano_fonte_recursos': 'int', 'fonte_recursos': 'str', 'funcao': 'int', 'subfuncao': 'int', 'exercicio': 'int', 'mes_referencia': 'int', 'educacao_saude': 'int', 'data_referencia': 'str', 'entrada_msc': 'int', 'natureza_despesa': 'str', 'ano_inscricao': 'int', 'valor': 'float', 'natureza_conta': 'str', 'tipo_valor': 'str', 'complemento_fonte': 'int'}

In [18]:
#função pra baixar os dados da API
def baixar_dados(path_raiz:str,tipo:str,mes:str,ano:str,id_ente:str)->None:
    periodo_lista = ['beginning_balance','period_change','ending_balance']
    #selecionado os parametros de clase da conta.
    if tipo == 'msc_patrimonial':
        inicio = 1
        fim = 5
    elif tipo == 'msc_orcamentaria':
        inicio = 5
        fim = 7
    else:
        inicio = 7
        fim = 9
    for i in range(inicio,fim):
        for periodo in periodo_lista:
            url = f'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/{tipo}?id_ente={id_ente}&an_referencia={ano}&me_referencia={mes}&co_tipo_matriz=MSCC&classe_conta={i}&id_tv={periodo}'
            resp = r.get(url)
            data = json.loads(resp.content)
            if(not(data["items"] == [])):
                arquivo = f'{path_raiz}\\{tipo}_classe{i}_{periodo}.csv'
                df = json_normalize(data["items"])
                #Conversão do arquivo para csv
                df.to_csv(arquivo,sep =';', index=False)

#função para juntar todos os arquivos de classe
def merge(arquivo:str, arquivos:list, path_de_saida:str,data_types_dict:dict)->None:
    df = pd.read_csv(arquivo, sep=';')
    os.remove(arquivo)
    for parte in arquivos:
        df2 = pd.read_csv(parte, sep=';')
        if(not(df2.empty)):
            df = pd.concat([df, df2], ignore_index=True)
            os.remove(parte)
    df = df.fillna(-1)
    df = df.astype(data_types_dict)
    df['valor'] = df['valor'].apply(lambda x: round(x,2))
    df.to_csv(path_de_saida,sep =';', decimal=',', index=False)
    
def merge_dos_arquivos(arquivo:str, arquivos:list, path_de_saida:str)->None:
    df = pd.read_csv(arquivo, sep=';')
    os.remove(arquivo)
    for parte in arquivos:
        df2 = pd.read_csv(parte, sep=';')
        if(not(df2.empty)):
            df = pd.concat([df, df2], ignore_index=True)
            os.remove(parte)
    df.to_csv(path_de_saida,sep =';', decimal=',', index=False)
    
#função para listar os Arquivos baixados    
def listar_path(comeco:str,arquivos_csv:str)->list:
    lista_caminho = []
    for subdir, _, files in os.walk(arquivos_csv,topdown=True):
               for file in files:
                    if file.startswith(comeco) and file.endswith('.csv'):
                        caminho_dos_arquivos =  subdir + "\\" + file
                        lista_caminho.append(caminho_dos_arquivos)
    return lista_caminho



In [27]:
# Execução do algoritmo e passagem de parametros.
raiz = 'F:\\CIC\\17 - Estagiários\\Rafael_Pereira\\MSC' # Alterar a raiz do caminho
lista_categoria = ('msc_patrimonial','msc_orcamentaria','msc_controle')
mes_referencia = ('01','02') # adicionar mais meses
ano_referencia = ('2022','2021') # adicionar outros anos
id_entidade = ('3301009',) # adicionar mais entidades
todas_combinacoes = tuple(product(lista_categoria, mes_referencia, ano_referencia,id_entidade))
for dados in todas_combinacoes:
    baixar_dados(raiz,dados[0],dados[1],dados[2],dados[3])
    lista = listar_path(f'{dados[0]}_',raiz)
    # o nome do arquivo de saida é a combinação do id da entidade pesquisada, a categoria pesquisada e a data da pesquisa
    if dados[0] == 'msc_patrimonial':
        merge(lista[0],lista[1:],f'{raiz}\\1{dados[0]}_{dados[1]}-{dados[2]}.csv',data_types_dict_patrimonial)
    elif dados[0] == 'msc_orcamentaria':
        merge(lista[0],lista[1:],f'{raiz}\\1{dados[0]}_{dados[1]}-{dados[2]}.csv',data_types_dict_orcamentaria)
    else:
        merge(lista[0],lista[1:],f'{raiz}\\1{dados[0]}_{dados[1]}-{dados[2]}.csv',data_types_dict_controle)
#juntando os dados em arquivos por classse
for dados in lista_categoria:    
    lista=listar_path(f'1{dados}_',raiz)
    merge_dos_arquivos(lista[0],lista[1:],f'{raiz}\\{dados}.csv')
